In [ ]:
from Bio import SeqIO, AlignIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
import subprocess
import tempfile
import os
import re
from io import StringIO, BytesIO
from bokeh.io import output_notebook, push_notebook
from bokeh.layouts import column, gridplot
from bokeh.plotting import figure, show
from bokeh.models.tools import BoxZoomTool
from bokeh.models.tickers import FixedTicker
from bokeh.models import Range1d, LinearAxis, BoxAnnotation
from bokeh.models.callbacks import CustomJS
from bokeh.resources import INLINE, CDN
from bokeh.embed import file_html
import numpy as np
from util import grouper
from align import align_mafft, read_ab1, trim_to_ref

In [ ]:
%load_ext autoreload
%autoreload 2
import plotting, align

In [ ]:
output_notebook(resources=INLINE)

In [ ]:
ref_seq = "GCGAACAGTTCGGCTGGCGCGAGCCCCTGATGCTCTTCGTCCAGATCATCCTGATCGACAAGACCGGCTTCCATCCGAGTACGTGCTCGCTCGATGCGATGTTTCGCTTGGTGGTCGAATGGGCAGGTAGCCGGATCAAGCGTATGCAGCCGCCGCATTGCATCAGCCATGATGGATACTTTCTCGGCAGGAGCAAGGTGAGATGACAGGAGATCCTGCCCCGGCACTTCGCCCAATAGCAGCCAGTCCCTTCCCGCTTCAGTGACAACGTCGAGCACAGCTGCGCAAGGAACGCCCGTCGTGGCCAGCCACGATAGCCGCGCTGCCTCGTCCTGCAGTTCATTCAGGGCACCGGACAGGTCGGTCTTGACAAAAAGAACCGGGCGCCCCTGCGCTGACAGCCGGAACACGGCGGCATCAGAGCAGCCGATTGTCTGTTGTGCCCAGTCATAGCCGAATAGCCTCTCCACCCAAGCGGCCGGAGAACCTGCGTGCAATCCATCTTGTTCAATCATGCGAAACGATCCTCATCCTGTCTCTTGATCAGATCTTGATCCCCTGCGCCATCAGATCCTTGGCGGCAAGAAAGCCATCCAGTTTACTTTGCAGGGCTTCCCAACCTTACCAGAGGGCGCCCCAGCTGGCAATTCCGACGTCTAAGAAACCATTATTATCATGACATTAACCTATAAAAATAGTCCTCCGCTTATTGATATGCGTCTTCACCACAGCTAACACCACGTCGTCCCTATCTGCTGCCCTAGGTCTATGAGTGGTTGCTGGATAACTTTACGGGCATGCATAAGGCTCGTATAATATATTCAGGGAGACCACAACGGTTTCCCTCTACAAATAATTTTGTTTAACTTTAGTAAGGAGTCTAGACCATGAGTAAAGGCGAAGAATTGTTCACTGGCGTGGTACCGATCCTGGTAGAACTGGATGGCGACGTTAATGGTCACAAGTTCAGCGTTAGTGGAGAGGGTGAAGGTGATGCGACCTATGGCAAACTGACCCTGAAGCTGATCTGCACAACCGGCAAGCTGCCTGTTCCTTGGCCGACACTGGTTACAACGCTGGGCTATGGCGTACAATGTTTCGCACGGTACCCGGACCACATGAAGCAACATGACTTCTTCAAGAGCGCTATGCCTGAAGGCTATGTCCAAGAAAGGACTATCTTCTTCAAAGACGACGGCAATTACAAGACACGGGCCGAAGTCAAATTCGAAGGCGATACGCTGGTCAACAGAATCGAGCTGAAAGGCATCGACTTCAAGGAAGATGGCAACATCCTGGGCCATAAACTGGAATATAATTATAACAGTCATAATGTGTATATCACCGCTGACAAACAAAAGAATGGCATCAAGGCCAACTTCAAAATCAGACATAACATCGAAGATGGAGGTGTTCAACTGGCAGACCACTACCAACAAAATACTCCGATCGGCGATGGCCCGGTGCTGCTGCCGGATAACCATTATCTGAGTTATCAAAGTAAGCTGAGCAAGGATCCGAACGAAAAAAGAGATCATATGGTTCTGCTGGAATTCGTAACGGCCGCGGGCATCACGCATGGCATGGACGAGCTGTATAAAAGTAAAGGAGAAGAACTTTTCACTGGAGTTGTCCCAATTCTTGTTGAATTAGATGGTGATGTTAATGGGCACAAATTTTCTGTCAGTGGAGAGGGTGAAGGTGATGCAACATACGGAAAACTTACCCTTAAATTTATTTGCACTACTGGAAAACTACCTGTTCCATGGCCAACACTTGTCACTACTCTCACTTGGGGTGTTCAATGCTTTGCAAGATACCCAGATCATATGAAACAGCATGACTTTTTCAAGAGTGCCATGCCCGAAGGTTATGTACAGGAAAGAACTATATTTTTCAAAGATGACGGGAACTACAAGACACGTGCTGAAGTCAAGTTTGAAGGTGATACCCTTGTTAATAGAATCGAGTTAAAAGGTATTGATTTTAAAGAAGATGGAAACATTCTTGGACACAAATTGGAATACAACTACATCTCAGACAATGTATACATCACGGCAGACAAACAAAAGAATGGAATCAAAGCTAACTTCAAAATTAGACACAACATTGAAGATGGAGGCGTTCAACTAGCAGACCATTATCAACAAAATACTCCAATTGGCGATGGCCCTGTCCTTTTACCAGACAACCATTACCTGTCCACACAATCTAAGCTTTCGAAAGATCCCAACGAAAAGAGAGACCACATGGTCCTTCTTGAGTTTGTAACAGCTGCTGGGATTACACATGGCATGGATGAACTATACAAAtaaATGTCCAGACCTGCAGGCATGCAAGCTCTAGAGGCATCAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTTGAAATGCACCAAAAACTCGTAAAAGCTCTGATGTATCTATCTTTTTTACACCGTTTTCATCTGTGCATATGGACAGTTTTCCCTTTGATATCTAACGGTGAACAGTTGTTCTACTTTTGTTTGTTAGTCTTGATGCTTCACTGATAGATACAAGAGCCATAAGAACCTCAGATCCTTCCGTATTTAGCCAGTATGTTCTCTAGTGTGGTTCGTTGTTTTTGCGTGAGCCATGAGAACGAACCATTGAGATCATGCTTACTTTGCATGTCACTCAAAAATTTTGCCTCAAAACTGGTGAGCTGAATTTTTGCAGTTAAAGCATCGTGTAGTGTTTTTCTTAGTCCGTTACGTAGGTAGGAATCTGATGTAATGGTTGTTGGTATTTTGTCACCATTCATTTTTATCTGGTTGTTCTCAAGTTCGGTTACGAGATCCATTTGTCTATCTAGTTCAACTTGGAAAATCAACGTATCAGTCGGGCGGCCTCGCTTATCAACCACCAATTTCATATTGCTGTAAGTGTTTAAATCTTTACTTATTGGTTTCAAAACCCATTGGTTAAGCCTTTTAAACTCATGGTAGTTATTTTCAAGCATTAACATGAACTTAAATTCATCAAGGCTAATCTCTATATTTGCCTTGTGAGTTTTCTTTTGTGTTAGTTCTTTTAATAACCACTCATAAATCCTCATAGAGTATTTGTTTTCAAAAGACTTAACATGTTCCAGATTATATTTTATGAATTTTTTTAACTGGAAAAGATAAGGCAATATCTCTTCACTAAAAACTAATTCTAATTTTTCGCTTGAGAACTTGGCATAGTTTGTCCACTGGAAAATCTCAAAGCCTTTAACCAAAGGATTCCTGATTTCCACAGTTCTCGTCATCAGCTCTCTGGTTGCTTTAGCTAATACACCATAAGCATTTTCCCTACTGATGTTCATCATCTGAGCGTATTGGTTATAAGTGAACGATACCGTCCGTTCTTTCCTTGTAGGGTTTTCAATCGTGGGGTTGAGTAGTGCCACACAGCATAAAATTAGCTTGGTTTCATGCTCCGTTAAGTCATAGCGACTAATCGCTAGTTCATTTGCTTTGAAAACAACTAATTCAGACATACATCTCAATTGGTCTAGGTGATTTTAATCACTATACCAATTGAGATGGGCTAGTCAATGATAATTACTAGTCCTTTTCCTTTGAGTTGTGGGTATCTGTAAATTCTGCTAGACCTTTGCTGGAAAACTTGTAAATTCTGCTAGACCCTCTGTAAATTCCGCTAGACCTTTGTGTGTTTTTTTTGTTTATATTCAAGTGGTTATAATTTATAGAATAAAGAAAGAATAAAAAAAGATAAAAAGAATAGATCCCAGCCCTGTGTATAACTCACTACTTTAGTCAGTTCCGCAGTATTACAAAAGGATGTCGCAAACGCTGTTTGCTCCTCTACAAAACAGACCTTAAAACCCTAAAGGCTTAAGTAGCACCCTCGCAAGCTCGGGCAAATCGCTGAATATTCCTTTTGTCTCCGACCATCAGGCACCTGAGTCGCTGTCTTTTTCGTGACATTCAGTTCGCTGCGCTCACGGCTCTGGCAGTGAATGGGGGTAAATGGCACTACAGGCGCCTTTTATGGATTCATGCAAGGAAACTACCCATAATACAAGAAAAGCCCGTCACGGGCTTCTCAGGGCGTTTTATGGCGGGTCTGCTATGTGGTGCTATCTGACTTTTTGCTGTTCAGCAGTTCCTGCCCTCTGATTTTCCAGTCTGACCACTTCGGATTATCCCGTGACAGGTCATTCAGACTGGCTAATGCACCCAGTAAGGCAGCGGTATCATCAACAGGCTTACCCGTCTTACTGTCAACCAGACCCGCCAGGATTCTCATGCATAAAAAACGCCCGGCGGCAACCGAGCGTTCTGAATTAATTAATCATCGCGACTGCAGAGATCCCAGAGTCCTTATTAGAAGAACTCGTCAAGAAGGCGATAGAAGGCGATGCGCTGCGAATCGGGAGCGGCGATACCGTAAAGCACGAGGAAGCGGTCAGCCCATTCGCCGCCAAGCTCTTCAGCAATATCACGGGTAGCCAACGCTATGTCCTGATAGCGGTCCGCCACACCCAGCCGGCCACAGTCGATGAATCCAGAAAAGCGGCCATTTTCCACCATGATATTCGGCAAGCAGGCATCGCCATGGGTCACGACGAGATCCTCGCCGTCGGGCATGCGCGCCTTGAGCCTG"

# Chromatogram

In [ ]:
# show_chromatogram_alignment(['testseq/242a7-ITS4.ab1', 'testseq/243a7-ITS4.ab1', 'testseq/252a7-ITS4.ab1', 'testseq/253a7-ITS4.ab1'], ref_seq)
plot = plotting.show_chromatogram_alignment(
    ["testseq/242a7-ITS4.ab1", "testseq/243a7-ITS4.ab1", "testseq/252a7-ITS4.ab1"],
    ref_seq,
)

In [ ]:
show(plot)

In [ ]:
with open("test.html", "w") as f:
    f.write(file_html(plot, INLINE, "Test Plot"))

In [ ]:
# fix correspondence between overview boxes and detail range (need to round to integer positions!)
# TODO: unrolled/wrapped chromatograph viewer
# figure out overview margins
# TODO: plot overview/detail EACH AS ONE PLOT (stacking in y-axis)
# reflow static html on load/window resize (for both combined and unrolled chromatograph viewer)